# Wrangling and cleaning data for geospatial plotting for New York City Bike 2022 

## Loading Libraries and data set

In [ ]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from dotenv import load_dotenv 
import os  
load_dotenv() 

True

In [40]:
# Get the full file path from .env
citibike_weather_path = os.getenv("CITIBIKE_2022_WEATHER")

# Read the CSV directly
citibike_weather_df = pd.read_csv(citibike_weather_path)

C:\Users\analy\AppData\Local\Temp\ipykernel_1336\4179577055.py:5: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  citibike_weather_df = pd.read_csv(citibike_weather_path)


In [5]:
citibike_weather_df.columns

Index(['date', 'month', 'weekday', 'tavg', 'prcp', 'ride_id', 'member_casual',
       'rideable_type', 'started_at', 'ended_at', 'duration', 'start_hour',
       'end_hour', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'value'],
      dtype='object')

In [6]:
citibike_weather_df.shape

(29838166, 22)

In [7]:
citibike_weather_df.dtypes

date                   object
month                   int64
weekday                object
tavg                  float64
prcp                  float64
ride_id                object
member_casual          object
rideable_type          object
started_at             object
ended_at               object
duration               object
start_hour              int64
end_hour                int64
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
value                   int64
dtype: object

In [8]:
df_nan= citibike_weather_df.isna().sum()
df_nan

date                      0
month                     0
weekday                   0
tavg                      0
prcp                      0
ride_id                   0
member_casual             0
rideable_type             0
started_at                0
ended_at                  0
duration                  0
start_hour                0
end_hour                  0
start_station_name       49
start_station_id         49
end_station_name      69884
end_station_id        69884
start_lat                 0
start_lng                 0
end_lat               37223
end_lng               37223
value                     0
dtype: int64

## Data cleaning Plan
### Problem: The data are devided into 4 groups:

Group 1: Raws with starting, ending station names and GPS coordinates are missing.i.e complete missing information to answer the geospatial questions. those raws must be cleaned out.

Group 2: missing satrting station name but available end station name. those wil be imputed with "missing start" and expected to be less than 49.

Group 3: missing end station names but available coordinates. those can be restored from the coordinates.

Group 4: Available starting station but missing end station name and coordinates. those should be imputed with "missing end". Goal: Keep these in the dataset so your total trip counts remain accurate, even if their destination is unknown.

### Cleaning Steps:

#### Step 1: The Purge — Drop rows where everything is NaN
Remove rows that have no usable information at all (e.g., missing both station names and GPS). This keeps the dataset clean from completely empty entries.

#### Step 2: Tagging Irrecoverable Data
Identify rows where the end station has no name AND incomplete GPS.
These rows cannot be recovered later, so they are tagged with a clear label (“Missing GPS and Name”) instead of being dropped.
This preserves them for counting but marks them as unusable for recovery.
### Recovering missing names through the coordinates information
#### Step 3: Build a Recovery Table (GPS → Station Name)
Create a compact reference table that contains only valid end station rows, each with a unique pair of coordinates and its correct station name.
This table acts as a clean lookup source for restoring missing names later.

#### Step 4: Attach Recovered Names via Merge
Merge the recovery table back into the main dataset using the end‑station coordinates.
This adds a temporary column containing the correct station name wherever a match exists.
Because merging is vectorized, it handles millions of rows efficiently.

#### Step 5: Finalize the Restoration
Fill in the missing end station names using the recovered values from the merge.
After the restoration is complete, remove the temporary helper column to return the dataset to its original structure.

#### Final cleanup: Any remaining NaNs get a general "Unknown" label
citibike_weather_df['end_station_name'] = citibike_weather_df['end_station_name'].fillna("Unknown Station")

Note: Station id cannot be taken as name alternative because every station name might have several id.


## Cleaning and Imputing missing values

In [41]:
# Step 1: Drop rows where everything is NaN
# Create a list with the columns of interest
NaN_cols = ['start_station_name', 'end_station_name', 'end_lat']  

# Drop a row only if all of those columns are NaN
citibike_weather_df.dropna(subset=NaN_cols, how='all', inplace=True) 

citibike_weather_df.shape # to count how many rows were dropped

(29838166, 22)

no raws where dropped. that means that all missing values in end station has starting station. 

In [43]:
# Step 2: Tagging Irrecoverable Data
# 1- Identify rows with no Name AND no GPS (creats Bolean_mask)
no_info_mask = (
    citibike_weather_df['end_station_name'].isna()
    & (citibike_weather_df['end_lat'].isna() | citibike_weather_df['end_lng'].isna())
)
# 2- Tag them so they are included in counts but clearly labeled
citibike_weather_df.loc[no_info_mask,'end_station_name'] = "Missing GPS and Name"

In [44]:
df_nan2= citibike_weather_df.isna().sum()
df_nan2

date                      0
month                     0
weekday                   0
tavg                      0
prcp                      0
ride_id                   0
member_casual             0
rideable_type             0
started_at                0
ended_at                  0
duration                  0
start_hour                0
end_hour                  0
start_station_name       49
start_station_id         49
end_station_name      32661
end_station_id        69884
start_lat                 0
start_lng                 0
end_lat               37223
end_lng               37223
value                     0
dtype: int64

69,884 - 32,661 = 37,223 end station name were taged as "Missing GPS and Name". exactly matches the missing number of coordinates.

32,661 end station name are missing but has coordinates. those can be recovered in the next steps



In [54]:
citibike_weather_df.dtypes


date                    object
month                    int64
weekday                 object
tavg                   float64
prcp                   float64
ride_id                 object
member_casual           object
rideable_type           object
started_at              object
ended_at                object
duration                object
start_hour               int64
end_hour                 int64
start_station_name      object
start_station_id        object
end_station_name        object
end_station_id          object
start_lat              float64
start_lng              float64
end_lat                float64
end_lng                float64
value                    int64
start_lat2             float64
start_lng2             float64
start_station_name2     object
dtype: object

### Restoring missing start station names


In [45]:
# round the coordinates numbers to avoid unmatching numbers after the merge (5 decimals ensures aroun 1 meter accurecy)
citibike_weather_df['start_lat'] = citibike_weather_df['end_lat'].round(5)
citibike_weather_df['start_lng'] = citibike_weather_df['end_lng'].round(5)
citibike_weather_df['end_lat'] = citibike_weather_df['end_lat'].round(5)
citibike_weather_df['end_lng'] = citibike_weather_df['end_lng'].round(5)

In [46]:
# Step 3A — Build a reference table (unique coordinates → station name)
ref_start_df = (
    citibike_weather_df
    .dropna(subset=['start_station_name', 'start_lat', 'start_lng'])
    .drop_duplicates(subset=['start_lat', 'start_lng'])
    [['start_lat', 'start_lng', 'start_station_name']]
)

In [47]:
# rename the headers to avoid replacing unmatched raws after merging
ref_start_df = ref_start_df.rename(columns={
    'start_station_name': 'start_station_name2',
    'start_lat': 'start_lat2',
    'start_lng': 'start_lng2'
})

In [48]:
ref_start_df.head()

,start_lat2,start_lng2,start_station_name2
0,40.70764,-73.96842,S 4 St & Wythe Ave
1,40.81336,-73.95646,Lexington Ave & E 120 St
3,40.72318,-73.99480,Cleveland Pl & Spring St
4,40.79615,-73.94782,Central Park North & Adam Clayton Powell Blvd
6,40.72020,-73.98998,E 2 St & 2 Ave


In [49]:
ref_start_df.shape


(2434, 3)

In [50]:
# Step 4A — Merge the recovery table back into the main DataFrame
citibike_weather_df = citibike_weather_df.merge(
    ref_start_df,
    left_on=['start_lat', 'start_lng'],
    right_on=['start_lat2', 'start_lng2'],
    how='left'
)

In [51]:
#Step 5A — Fill missing names using the recovered values
citibike_weather_df['start_station_name'] = (
    citibike_weather_df['start_station_name']
    .fillna(citibike_weather_df['start_station_name2'])
)

In [52]:
citibike_weather_df.columns

Index(['date', 'month', 'weekday', 'tavg', 'prcp', 'ride_id', 'member_casual',
       'rideable_type', 'started_at', 'ended_at', 'duration', 'start_hour',
       'end_hour', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'value', 'start_lat2', 'start_lng2',
       'start_station_name2'],
      dtype='object')

In [53]:
df_nan3= citibike_weather_df.isna().sum()
df_nan3

date                       0
month                      0
weekday                    0
tavg                       0
prcp                       0
ride_id                    0
member_casual              0
rideable_type              0
started_at                 0
ended_at                   0
duration                   0
start_hour                 0
end_hour                   0
start_station_name         0
start_station_id          49
end_station_name       32661
end_station_id         69884
start_lat              37223
start_lng              37223
end_lat                37223
end_lng                37223
value                      0
start_lat2             37223
start_lng2             37223
start_station_name2    37223
dtype: int64

In [38]:
citibike_weather_df.dtypes

date                    object
month                    int64
weekday                 object
tavg                   float64
prcp                   float64
ride_id                 object
member_casual           object
rideable_type           object
started_at              object
ended_at                object
duration                object
start_hour               int64
end_hour                 int64
start_station_name      object
start_station_id        object
end_station_name        object
end_station_id          object
start_lat              float64
start_lng              float64
end_lat                float64
end_lng                float64
value                    int64
start_station_name2     object
dtype: object

merging the reference df with the original df and restoring the missing 49 start station names has resulted in full restoring where 0 missing start station name has remained accroding to the isna table above.

However, 37223 missing raws have apeared in the start coordinates after the merge. those are NaN data that were exposed after the merge.

as a result we see the missing values now and they are equall to the missing end station coordinates, which makes sense.

for that i will do for those missing start station names tagging as i did in step 2 for the end station names to filter them later.
i am not deleting the raws because the station names are available and they still considered valid rides to be counted.

In [61]:
# repeat Step 2: Tagging Irrecoverable Data for the missing start station coordinates
# 1 — Identify rows with missing start coordinates
missing_gps_mask = (
    citibike_weather_df['start_lat'].isna() |
    citibike_weather_df['start_lng'].isna()
)

# 2 — Tag the coordinate columns directly
citibike_weather_df.loc[missing_gps_mask, 'start_lat'] = "Missing GPS"
citibike_weather_df.loc[missing_gps_mask, 'start_lng'] = "Missing GPS"

In [62]:
df_nan4= citibike_weather_df.isna().sum()
df_nan4

date                       0
month                      0
weekday                    0
tavg                       0
prcp                       0
ride_id                    0
member_casual              0
rideable_type              0
started_at                 0
ended_at                   0
duration                   0
start_hour                 0
end_hour                   0
start_station_name         0
start_station_id          49
end_station_name       32661
end_station_id         69884
start_lat                  0
start_lng                  0
end_lat                37223
end_lng                37223
value                      0
start_lat2             37223
start_lng2             37223
start_station_name2    37223
dtype: int64

In [ ]:
# Save the so far cleaned and wrangled dataset to load it again for the next steps that require big ram
# Load the new path from .env
citibike_weather_path2 = os.getenv("CITIBIKE_2022_WEATHER2")

# Save the cleaned dataframe
citibike_weather_df.to_csv(citibike_weather_path2, index=False)

In [ ]:
# Save the so far cleaned and wrangled dataset to load it again for the next steps that require big ram
# Load the new path from .env
citibike_weather_path2 = os.getenv("CITIBIKE_2022_WEATHER2")

# Save the cleaned dataframe in chunks to avoid memory issues
# Using chunksize parameter to process the dataframe in smaller pieces
chunk_size = 100000  # Adjust this value based on your available memory
for i, chunk in enumerate(np.array_split(citibike_weather_df, len(citibike_weather_df) // chunk_size + 1)):
    if i == 0:
        # Write header for the first chunk
        chunk.to_csv(citibike_weather_path2, index=False, mode='w')
    else:
        # Append without header for subsequent chunks
        chunk.to_csv(citibike_weather_path2, index=False, mode='a', header=False)

### Problem
Memory error if i try to continue doing the above steps 3 to 5 for the end station missing names.
also memory error if i try to save as above.

### Restoring missing end station names

In [76]:
# Step 3B — Build a reference table (unique end coordinates → end station name)
ref_end_df = (
    citibike_weather_df
    .dropna(subset=['end_station_name', 'end_lat', 'end_lng'])
    .drop_duplicates(subset=['end_lat', 'end_lng'])
    [['end_lat', 'end_lng', 'end_station_name']]
)

In [77]:
# rename the headers to avoid replacing unmatched raws after merging
ref_end_df = ref_end_df.rename(columns={
    'end_station_name': 'end_station_name2',
    'end_lat': 'end_lat2',
    'end_lng': 'end_lng2'
})

In [78]:
ref_end_df.head()

,end_lat2,end_lng2,end_station_name2
0,40.70764,-73.96842,Kent Ave & S 11 St
1,40.81336,-73.95646,Amsterdam Ave & W 125 St
3,40.72318,-73.99480,Mott St & Prince St
4,40.79615,-73.94782,E 110 St & Madison Ave
6,40.72020,-73.98998,Allen St & Rivington St


In [ ]:
# Step 4B — Merge to attach recovered end station names
citibike_weather_df = citibike_weather_df.merge(
    ref_end_df,
    left_on=['end_lat', 'end_lng'],
    right_on=['end_lat2', 'end_lng2'],
    how='left',
    
)

In [17]:
# Step 5B — Fill missing end station names
citibike_weather_df['end_station_name'] = (
    citibike_weather_df['end_station_name']
    .fillna(citibike_weather_df['end_station_name_end_recovered'])
)

In [36]:
citibike_weather_df.head()

,date,month,weekday,tavg,prcp,ride_id,member_casual,rideable_type,started_at,ended_at,...,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,value,end_station_name_end_recovered,start_station_name_recovered
0,2022-01-01,1,Saturday,11.6,19.3,4A091640AEC83471,member,classic_bike,2022-01-01 16:01:22.082,2022-01-01 16:05:11.986,...,5204.05,Kent Ave & S 11 St,5062.01,40.712859,-73.965903,40.707645,-73.968415,1,Kent Ave & S 11 St,S 4 St & Wythe Ave
1,2022-01-01,1,Saturday,11.6,19.3,7A560D6287C00126,member,classic_bike,2022-01-01 14:07:04.344,2022-01-01 14:31:57.500,...,7652.04,Amsterdam Ave & W 125 St,7800.03,40.801307,-73.939817,40.813358,-73.956461,1,Amsterdam Ave & W 125 St,Lexington Ave & E 120 St
2,2022-01-01,1,Saturday,11.6,19.3,4F6F726B809C15E4,member,classic_bike,2022-01-01 12:02:26.943,2022-01-01 12:14:13.555,...,7652.04,Amsterdam Ave & W 125 St,7800.03,40.801307,-73.939817,40.813358,-73.956461,1,Amsterdam Ave & W 125 St,Lexington Ave & E 120 St
3,2022-01-01,1,Saturday,11.6,19.3,3BC10DC94FD79955,member,classic_bike,2022-01-01 22:53:23.508,2022-01-01 23:18:42.881,...,5492.05,Mott St & Prince St,5561.04,40.722104,-73.997249,40.723180,-73.994800,1,Mott St & Prince St,Cleveland Pl & Spring St
4,2022-01-01,1,Saturday,11.6,19.3,FE4CE7F1CFFD7952,member,classic_bike,2022-01-01 08:04:51.755,2022-01-01 08:08:07.290,...,7617.07,E 110 St & Madison Ave,7587.14,40.799484,-73.955613,40.796154,-73.947821,1,E 110 St & Madison Ave,Central Park North & Adam Clayton Powell Blvd


In [37]:
citibike_weather_df.columns 

Index(['date', 'month', 'weekday', 'tavg', 'prcp', 'ride_id', 'member_casual',
       'rideable_type', 'started_at', 'ended_at', 'duration', 'start_hour',
       'end_hour', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'value', 'end_station_name_end_recovered',
       'start_station_name_recovered'],
      dtype='object')

In [21]:
citibike_weather_df.dtypes

date                               object
month                               int64
weekday                            object
tavg                              float64
prcp                              float64
ride_id                            object
member_casual                      object
rideable_type                      object
started_at                         object
ended_at                           object
duration                           object
start_hour                          int64
end_hour                            int64
start_station_name                 object
start_station_id                   object
end_station_name                   object
end_station_id                     object
start_lat                         float64
start_lng                         float64
end_lat                           float64
end_lng                           float64
value                               int64
start_station_name_recovered       object
end_station_name_end_recovered    

In [35]:
df_nan4= citibike_weather_df.isna().sum()
df_nan4

date                                  0
month                                 0
weekday                               0
tavg                                  0
prcp                                  0
ride_id                               0
member_casual                         0
rideable_type                         0
started_at                            0
ended_at                              0
duration                              0
start_hour                            0
end_hour                              0
start_station_name                   49
start_station_id                     49
end_station_name                      0
end_station_id                    69884
start_lat                             0
start_lng                             0
end_lat                           37223
end_lng                           37223
value                                 0
end_station_name_end_recovered    69884
start_station_name_recovered         49
dtype: int64

In [24]:
# Final cleanup: Any remaining NaNs get a general "Unknown" label
# citibike_weather_df['end_station_name'] = citibike_weather_df['end_station_name'].fillna("Unknown Station")

In [26]:
# Drop the helper columns 
# citibike_weather_df.drop(columns=['start_station_name_recovered'], inplace=True)